In [1]:
from xml.etree import ElementTree
from string import ascii_lowercase
import os
import pandas as pd

In [2]:
# parsing xml files to get bible as a text file
for lang in ['English', 'German', 'French', 'Polish', 'Czech', 'Slovak', 'Dutch', 'Finnish', 'Hungarian']:
    if f'{lang}.txt' in os.listdir('data/parsed'):
        continue

    root = ElementTree.parse(f'data/bibles/{lang}.xml')
    print(lang)
    with open(f'data/parsed/{lang}.txt', 'w') as out:
        for line in root.iter('seg'):
            try:
                out.write(line.text.strip() + '\n')
            except AttributeError:
                pass


In [3]:
for path in os.listdir('data/parsed'):
    lang = path.split('.')[0]
    if f'{lang}.csv' in os.listdir('data/processed'):
        continue

    print(path)
    with open(f'data/parsed/{path}', 'r') as file:
        data = pd.DataFrame({'text': file.readlines(), 'lang': lang})

        # leaving only ascii letters
        data['letters'] = data.text.str.lower().str.strip()#.str.replace('(?![a-z]).', '', regex=True)
        data['letters'] = data.letters.map(lambda y: ''.join(filter(lambda x: x.isalpha(), y)))
        # making a vector of number of occurrences for each letter
        for letter in ascii_lowercase:
            data[letter] = data.letters.str.count(letter)

        # changing counts to probabilities
        lengths = data.letters.str.len()
        data.loc[:, 'a':].apply(lambda x: x / lengths, axis=0)

        data.to_csv(f'data/processed/{lang}.csv', index=False) # saving as csv